In [231]:
# Paweł Iwiński, Cezary Graban - NAI zjazd 3 - KNN
# Referencja 1: https://towardsdatascience.com/item-based-collaborative-filtering-in-python-91f747200fab
# Referencja 2: https://linuxhint.com/get-movie-information-raspberry-pi/
import sys
import os
!{sys.executable} -m pip install cinemagoer

In [232]:
import numpy as np
import pandas as pd
import imdb
from tmdbv3api import TMDb, Movie
from sklearn.neighbors import NearestNeighbors


In [233]:
# Load the data
df = pd.read_csv("NAI - dane.csv", header=None)
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,63,64,65,66,67,68,69,70,71,72
0,Paweł Czapiewski,Polowanie na Czerwony Październik,10,Rick & Morty,10,The Big Bang Theory,8,Braveheart,10,The Expanse,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Łukasz Cettler,Rick & Morty,10,Fight club,10,Ona,7,Whiplash,9,John Wick,...,Deadpool,8.0,Fantastyczne zwierzęta,4.0,Pięćdziesiąt twarzy black'a,3.0,365 dni,1.0,Wyznania Gejszy,9.0
2,Paweł Iwiński,Ojciec Chrzestny,10,South Park,9,Na wspólnej,3,The Room,5,Chłopaki z baraków,...,To,6.0,Przegryź to,2.0,Moneyball,6.0,Hostel,2.0,Złodziej tożsamości,3.0
3,Oktawian Filipkowski,Strażnicy Galaktyki,8,Strażnicy Galaktyki 2,9,Thor : Ragnarok,8,Fate/Apocrypha,10,Pianista,...,Kapitan Bomba,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Krzysztof Lewandowski,CONTRATIEMPO,10,Praktykant,10,Harry Potter,10,Nietykalni,10,Forrest Gump,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [234]:
# Transpose the dataframe
df = df.T
df.columns = df.iloc[0]
df = df[1:]
df

,Paweł Czapiewski,Łukasz Cettler,Paweł Iwiński,Oktawian Filipkowski,Krzysztof Lewandowski,Kamil Kornatowski,Dominik Pasymowski,Maciej Zakrzewski,Jakub Jabłoński,Stanisław Kibort,Marcin Kloczkowski,Wojciech Mierzejewski,Adam Kałuża,Cezary Graban,Sylwester Kąkol,Adam Jurkiewicz,Michał Cichowski
1,Polowanie na Czerwony Październik,Rick & Morty,Ojciec Chrzestny,Strażnicy Galaktyki,CONTRATIEMPO,Blade Runner,Rick & Morty,Między piekłem a niebem,Narcos,Django,Kapitan Bomba,Siedmiu Samurajów,Rick & Morty,Avatar,Jojo Rabbit,Na noże,Casino
2,10,10,10,8,10,10,8,10,10,10,10,10,10,8,10,8,9
3,Rick & Morty,Fight club,South Park,Strażnicy Galaktyki 2,Praktykant,Hydrozagadka,Harry Potter,Dzień świra,Rick & Morty,Rick & Morty,Star Wars,Popiół i Diament,Władca Pierścieni,South Park,Dunkierka,Parasite,Heat
4,10,10,9,9,10,8,10,10,10,10,8,10,10,9,10,8,10
5,The Big Bang Theory,Ona,Na wspólnej,Thor : Ragnarok,Harry Potter,Kraina Lodu,Star Wars,K-PAX,Game of Thrones,Star Wars,Forrest Gump,Prawo i Pięść,Kapitan Bomba,Rick & Morty,Bękarty Wojny,Diuna,Niezwykły przypadek Brnjamina Buttona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,NaN,3,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,7,NaN
69,NaN,365 dni,Hostel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wiedźmin,Avatar,NaN
70,NaN,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,8,NaN
71,NaN,Wyznania Gejszy,Złodziej tożsamości,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Forrest Gump,NaN


In [235]:
# Flatten the df in order to obtain lsit with users, ratings and movies
squash = []
for (columnName, columnData) in df.iteritems():
    squash.append(df[columnName])

df_per_user = []
for i in range(len(squash)):
    movies = squash[i][::2].reset_index(drop=True)
    scores = squash[i][1::2].reset_index(drop=True)
    concated = pd.merge(movies, scores, left_index=True, right_index=True)

    concated["user"] = movies.name
    concated.columns = ["movie", "rating", "user"]
    concated = concated.dropna()
    df_per_user.append(concated)

concat_df = pd.concat(df_per_user).reset_index(drop=True)
concat_df

,movie,rating,user
0,Polowanie na Czerwony Październik,10,Paweł Czapiewski
1,Rick & Morty,10,Paweł Czapiewski
2,The Big Bang Theory,8,Paweł Czapiewski
3,Braveheart,10,Paweł Czapiewski
4,The Expanse,7,Paweł Czapiewski
...,...,...,...
482,Inflitracja,7.5,Michał Cichowski
483,Choć goni nas czas,8,Michał Cichowski
484,Ojciec Chrzesny,8,Michał Cichowski
485,Friday,7,Michał Cichowski


In [236]:
# Get unique values
scores = concat_df[["user", "rating"]]
movies = concat_df["movie"].unique()
users = concat_df["user"].unique()

In [237]:
# Create df out of the unique values and assaign Nan to all cell values
data = pd.DataFrame(index=range(0,len(movies)),columns=users)
data.index = movies
data


,Paweł Czapiewski,Łukasz Cettler,Paweł Iwiński,Oktawian Filipkowski,Krzysztof Lewandowski,Kamil Kornatowski,Dominik Pasymowski,Maciej Zakrzewski,Jakub Jabłoński,Stanisław Kibort,Marcin Kloczkowski,Wojciech Mierzejewski,Adam Kałuża,Cezary Graban,Sylwester Kąkol,Adam Jurkiewicz,Michał Cichowski
Polowanie na Czerwony Październik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rick & Morty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
The Big Bang Theory,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Braveheart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
The Expanse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Inflitracja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Choć goni nas czas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ojciec Chrzesny,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Friday,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [238]:
# Fill the Nan values from above with concataned dataframe calculated previously
integer = 0
for rowIndex, row in data.iterrows(): #iterate over rows
    for columnIndex, value in row.items():
        if concat_df[(concat_df['movie'] == rowIndex) & (concat_df['user'] == columnIndex)]["rating"].values.size > 0:
            value = concat_df[(concat_df['movie'] == rowIndex) & (concat_df['user'] == columnIndex)]["rating"].values[0]
        else:
            value = 0
        data.loc[rowIndex, columnIndex] = value

df1 = data.copy()
df1

,Paweł Czapiewski,Łukasz Cettler,Paweł Iwiński,Oktawian Filipkowski,Krzysztof Lewandowski,Kamil Kornatowski,Dominik Pasymowski,Maciej Zakrzewski,Jakub Jabłoński,Stanisław Kibort,Marcin Kloczkowski,Wojciech Mierzejewski,Adam Kałuża,Cezary Graban,Sylwester Kąkol,Adam Jurkiewicz,Michał Cichowski
Polowanie na Czerwony Październik,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Rick & Morty,10,10,7,0,0,0,8,0,10,10,0,0,10,9,7,0,0
The Big Bang Theory,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Braveheart,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
The Expanse,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Inflitracja,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7.5
Choć goni nas czas,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8
Ojciec Chrzesny,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8
Friday,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7


In [239]:
# Copy both df for simplicity in the naming system.
df = df1.copy()
df1 = df.copy()

In [240]:
# Create a function for a movie recomendations for a given user
def recommend_movies(user, num_recommended_movies):
    """Recommend movies for a given user based on sorted list with movies.

    Args:
        user (str): User for which we want recommendations.
        num_recommended_movies (int): How many movies do we want for user.
    """
    recommended_movies = []
    for m in df[df[user] == 0].index.tolist():
        index_df = df.index.tolist().index(m)
        predicted_rating = df1.iloc[index_df, df1.columns.tolist().index(user)]
        recommended_movies.append((m, predicted_rating))

    sorted_rm = sorted(recommended_movies, key=lambda x: x[1], reverse=True)

    print("Recommended movies")
    rank = 1
    for recommended_movie in sorted_rm[:num_recommended_movies]:
        print("{}: {}".format(rank, recommended_movie[0]))
        rank = rank + 1
    print('\n')

    sorted_rm = sorted(recommended_movies, key=lambda x: x[1], reverse=False)
    print("Not recommended movies")
    rank = 1
    for recommended_movie in sorted_rm[:num_recommended_movies]:
        print("{}: {}".format(rank, recommended_movie[0]))
        rank = rank + 1


In [241]:
df1 = df.copy()
# Recomend a movie based on a KNN
def movie_recommender(user, num_neighbors, num_recommendation, metric="cosine", algorithm="brute"):
    """Function for movie recommendations that uses the KNN to obtain closest distance for the movie scores.

    Args:
        user (str): User for which we want recommendations.
        num_neighbors (int): Number of neighbours in KNN
        num_recommendation (int): Number of the recomendations that we want for the user.
        metric (str, optional): Matric for knn. Defaults to "cosine".
        algorithm (str, optional): Algorithm for knn. Defaults to "brute".
    """
    number_neighbors = num_neighbors

    knn = NearestNeighbors(metric=metric, algorithm=algorithm)
    knn.fit(df.values)
    distances, indices = knn.kneighbors(df.values, n_neighbors=number_neighbors)

    user_index = df.columns.tolist().index(user)

    for m, t in list(enumerate(df.index)):
        if df.iloc[m, user_index] == 0:
            sim_movies = indices[m].tolist()
            movie_distances = distances[m].tolist()

            if m in sim_movies:
                id_movie = sim_movies.index(m)
                sim_movies.remove(m)
                movie_distances.pop(id_movie)
            else:
                sim_movies = sim_movies[: num_neighbors - 1]
                movie_distances = movie_distances[: num_neighbors - 1]

            movie_similarity = [1 - x for x in movie_distances]
            movie_similarity_copy = movie_similarity.copy()
            nominator = 0

            for s in range(0, len(movie_similarity)):
                if df.iloc[sim_movies[s], user_index] == 0:
                    if len(movie_similarity_copy) == (number_neighbors - 1):
                        movie_similarity_copy.pop(s)
                    else:
                        movie_similarity_copy.pop(
                            s - (len(movie_similarity) - len(movie_similarity_copy))
                        )
                else:
                    nominator = (
                        nominator
                        + movie_similarity[s] * df.iloc[sim_movies[s], user_index]
                    )

            if len(movie_similarity_copy) > 0:
                if sum(movie_similarity_copy) > 0:
                    predicted_r = nominator / sum(movie_similarity_copy)
                else:
                    predicted_r = 0

            else:
                predicted_r = 0

            df1.iloc[m, user_index] = predicted_r
    recommend_movies(user, num_recommendation)


In [242]:
# Type user below, with number of neigbours and number of movie recommendations.
movie_recommender("Paweł Iwiński", 3, 5, metric="cosine", algorithm="brute")

Recommended movies
1: Parasite
2: John Wick
3: Kraina Lodu
4: Narodziny Gwiazdy
5: Terminal


Not recommended movies
1: Polowanie na Czerwony Październik
2: The Big Bang Theory
3: Braveheart
4: The Expanse
5: Dziennik Bridget Jones


In [243]:
# Type user below, with number of neigbours and number of movie recommendations.
movie_recommender("Paweł Iwiński", 3, 5, algorithm="ball_tree", metric="minkowski")

Recommended movies
1: Polowanie na Czerwony Październik
2: The Big Bang Theory
3: Braveheart
4: The Expanse
5: Kraina Lodu


Not recommended movies
1: Polowanie na Czerwony Październik
2: The Big Bang Theory
3: Braveheart
4: The Expanse
5: Kraina Lodu


In [244]:
# Type user below, with number of neigbours and number of movie recommendations.
movie_recommender("Paweł Iwiński", 3, 5, algorithm="auto", metric="cosine")

Recommended movies
1: Parasite
2: John Wick
3: Kraina Lodu
4: Narodziny Gwiazdy
5: Terminal


Not recommended movies
1: Polowanie na Czerwony Październik
2: The Big Bang Theory
3: Braveheart
4: The Expanse
5: Dziennik Bridget Jones


In [245]:
movie = "Avengers"

# define a function to print names from a list
def List_of_names(nameList):
    """Extract name and surname from the lsit with people.

    Args:
        nameList (list): List containing people working on a movie

    Returns:
        str: long string containing people.
    """
    names=''
    # for each person object, extracts name tag and append to our names string
    if nameList is None: return ''
    for i in nameList: names=names+'; '+str(i.get('name'))
    # returns final string shifted by 2 chars to manage initial " ;"
    return names[2:]


# initializes IMDb funtion and searches for our name
x= imdb.IMDb()
movies = x.search_movie(movie)

# if more film titles are matching search, ask user to refine search title
if len(movies) > 1:
    print('More films matching query:\n')
    print('Number | Film title')
    print('--------------------')
    id=0
    for i in movies:
        print(str(id)+'  |   '+i['title'])
        id +=1
    # Ask user to choos film mumber
    userInput=input("Please input film number: ")
    film=movies[int(userInput)]
    print()
else:
    # if only 1 film matches search, it is automatically selected
    film=movies[0]

filmID=film.movieID

# get film data
movie = x.get_movie(filmID)

More films matching query:

Number | Film title
--------------------
0  |   The Avengers
1  |   Marvel's Avengers
2  |   Avengers: United They Stand
3  |   Avengers: Endgame
4  |   Avengers: Infinity War
5  |   Avengers: Secret Wars
6  |   Avengers: Age of Ultron
7  |   Avengers: The Kang Dynasty
8  |   The Avengers
9  |   Passengers
10  |   The Avengers
11  |   Avengement
12  |   Captain America: The First Avenger
13  |   Tokyo Revengers
14  |   Avengers Assemble
15  |   Avenged
16  |   The Toxic Avenger
17  |   Avenger
18  |   Halloween 5: The Revenge of Michael Myers
19  |   The New Avengers



In [246]:
# print main film data
print('Title: ' + movie.get('title'))
print('IMDb ID: ' + str(filmID))
print()
print("Plot:" + str(movie.get("plot")))
print()
print('Cover URL: '+ str(movie.get('cover url')))
print()
print('Original title: '+movie.get('original title')+' | '+str(movie.get('genres')))
print()
print('Rating: ' + str(movie.get('rating'))+' (based on '+str(movie.get('votes'))+' votes)')
print()
print('Directors: '+List_of_names(movie.get('directors')))

Title: The Avengers
IMDb ID: 0848228

Plot:["Earth's mightiest heroes must come together and learn to fight as a team if they are going to stop the mischievous Loki and his alien army from enslaving humanity.", "Loki, the adopted brother of Thor, teams-up with the Chitauri Army and uses the Tesseract's power to travel from Asgard to Midgard to plot the invasion of Earth and become a king. The director of the agency S.H.I.E.L.D., Nick Fury, sets in motion project Avengers, joining Tony Stark a.k.a. the Iron Man; Steve Rogers, a.k.a. Captain America; Bruce Banner, a.k.a. The Hulk; Thor; Natasha Romanoff, a.k.a. Black Widow; and Clint Barton, a.k.a. Hawkeye, to save the world from the powerful Loki and the alien invasion.::Claudio Carvalho, Rio de Janeiro, Brazil", "S.H.I.E.L.D. has located the mysterious Tesseract device and the Army's super soldier Captain America. The Tesseract is actually a gateway to an entirely new world called Asgard. A mysterious being known as Loki arrives on ear